# VIRAT Clip Generator

## Proccess Object Annotation File 
File format: `*viratdata.objects.txt`

|Column| Type |Comment|
|---|:------|:--------|
|1| Object id|a unique identifier of an object track. Unique within a file.|
|2| Object duration | duration of the object track|
|3| Currnet frame|corresponding frame number|
|4| bbox lefttop x|horizontal x coordinate of the left top of bbox, origin is lefttop of the frame|
|5| bbox lefttop y|vertical y coordinate of the left top of bbox, origin is lefttop of the frame|
|6| bbox width|horizontal width of the bbox|
|7| bbox height|vertical height of the bbox|
|8| Object Type|object type|

Object Type ID (for column 8 above for object files)

ID | Type|
---|:----|
1| person
2| car              (usually passenger vehicles such as sedan, truck)
3| vehicles         (vehicles other than usual passenger cars. Examples include construction vehicles)
4| object           (neither car or person, usually carried objects)
5| bike, bicylces   (may include engine-powered auto-bikes)

In [1]:
import os
import cv2
import time
import random
import numpy as np
import pandas as pd

from tqdm import tqdm
from write_html import writeHTML
from utils.virat_utils import cut_off_frame, generate_focused_area_mask, generate_object_trajectory, Color

print(cv2.__version__)

3.3.1


In [2]:
## PARAMTERS
CHUNK  = 50
OFFSET = 150
OUTPUT_DIR             = './outputs'
EXAMPLE                = 'VIRAT_S_000207_00_000000_000045'
DEFAULT_PATH           = '/media/dat/dataset/VIRAT'

# Naming Convention for the image output file
# {0}: original video file name
# {1}: chunk number
# {2}: current frame
OUTPUT_FORMAT    = '{0}_{1}_{2}.png'

OBJECT_TYPES           = ['_', 'person', 'car', 'vehicles','object','bike']
SELECTED_OBJECT_TYPE   = [ 2, 3]  # only select car or vehicles
object_anno_fields     = ['object_id', 'object_duration', 'current_frame',
                          'left_top_x','left_top_y', 'width', 'height', 
                          'object_type']


In [3]:
DEFAULT_ANNOTATION_DIR = os.path.join(DEFAULT_PATH,'annotations')
DEFAULT_VIDEO_DIR      = os.path.join(DEFAULT_PATH,'videos_original')

if not os.path.isdir(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)
    
anno_path = os.path.join(DEFAULT_ANNOTATION_DIR, EXAMPLE + '.viratdata.objects.txt')
df        = pd.read_csv(anno_path, delim_whitespace =True, names=object_anno_fields)

# Group each objects appeared in the video by object id
separated_objects_by_id = df.groupby('object_id')

In [4]:
# Generate a list of _boxes for each objects appear in the video
object_bboxes = []
temp_frames   = []


for _, obj in separated_objects_by_id:
    
    object_type = obj['object_type'].values[0]

    if object_type not in SELECTED_OBJECT_TYPE:
        continue
        
    print("Found %s" % OBJECT_TYPES[object_type])    
    # Extract the bounding boxes
    upper_left_pts   = zip(obj['left_top_x'].values, obj['left_top_y'].values)
    width_height_lst = zip(obj['width'].values,      obj['height'].values)
    
    lower_right_pts  = [(px + w, py + h) 
                            for (px, py),(w, h) in zip(upper_left_pts, width_height_lst)]
    
    bbox_list        = [[(px1, py1), (px2, py2)] 
                            for px1, py1, (px2, py2) in zip(obj['left_top_x'].values, 
                                                              obj['left_top_y'].values, 
                                                              lower_right_pts)]
    if bbox_list:
        object_bboxes.append(bbox_list)
        temp_frames.append(obj)
  
# print("Number of annotated objects in the video %s"%len(object_bboxes))
grouped_objects = pd.concat(temp_frames).groupby('object_id')

Found car
Found car
Number of annotated objects in the video 2


In [6]:
# pick random object in grouped object, which satisfy this condition
curr = 0 
cut_off_idx = 0
max_tries   = 50
while curr <= max_tries:
    SELECTED_OBJECT =  random.choice(list(grouped_objects.groups)) # ID of object in the video
    OBJECT_IDX      = list(grouped_objects.groups).index(SELECTED_OBJECT)
    
    # Get object bounding boxes
    bboxes_of_object = object_bboxes[OBJECT_IDX]
    
    # If object is stationary for a duration (e.g car is parking), filter out
    cut_off_idx  = cut_off_frame(bboxes_of_object, duration=CHUNK)
    if cut_off_idx is not 0:
        break
        
    curr +=1 

if  cut_off_idx:
    bbox_arr    = np.asarray(bboxes_of_object[:cut_off_idx])
    num_chunk   = int(len(bbox_arr) / CHUNK)
    bbox_chunks = np.array_split(bbox_arr,num_chunk)
else:
    raise ValueError("All objects in this video are stationary. Please try other videos.")

# In this example, we only show the video when the selected object appears
appear_frames = grouped_objects.get_group(SELECTED_OBJECT)['current_frame'].values
start_frame   = np.min(appear_frames)
end_frame     = start_frame + cut_off_idx

# print("Number of required frames: %s, starting at %s"% (end_frame - start_frame, start_frame))

In [10]:
for group in grouped_objects.groups:
    print(group)

1
3


In [8]:
# For each frames, draw all the appear bounding boxes:
processed_frames = []


video_file = EXAMPLE.split('_')
video_file = "_".join(video_file[:3])
video_path = os.path.join(DEFAULT_VIDEO_DIR, EXAMPLE+'.mp4')

cap        = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise IOError("Please check video path %s" % video_path)

    
OUTPUT_FOLDER = os.path.join(OUTPUT_DIR, video_file)
if not os.path.isdir(OUTPUT_FOLDER):
    os.mkdir(OUTPUT_FOLDER)

# Generate a image mask, size of the clip's dimension
_, frame = cap.read()
fps               = cap.get(cv2.CAP_PROP_FPS)
empty_mask        = np.zeros_like(frame)
object_trajectory = generate_object_trajectory(empty_mask, bboxes_of_object, 
                                               color=Color.green, 
                                               opacity=60)


start = time.time()
print("Processing %s" % video_file)
for i, bbox_chunk in enumerate(tqdm(bbox_chunks)):
    focused_area_mask, top_pts, bot_pts =  generate_focused_area_mask(empty_mask, 
                                                                      bbox_chunk, 
                                                                      color=Color.yellow, 
                                                                      offset=OFFSET)
    
    CURR_SEQUENCE_DIR = os.path.join(OUTPUT_FOLDER, str(i))
    if not os.path.isdir(CURR_SEQUENCE_DIR):
        os.mkdir(CURR_SEQUENCE_DIR)

    captions         = []
    image_paths      = []
    
    for idx, (p1, p2) in enumerate(bbox_chunk):
        # Format file name
        filename   = OUTPUT_FORMAT.format(video_file, i, idx)
        saved_path = os.path.join(CURR_SEQUENCE_DIR, filename)
        
        # Get current frame from clip
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame + i*CHUNK + idx)
        _, frame = cap.read()
        
        # convert to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Draw bounding box of the object on current frame
        frame = cv2.rectangle(frame, (p1[0], p1[1]), (p2[0], p2[1]), color=Color.green, thickness=4)
        
        # Add object flow mask to current frame
        frame = cv2.addWeighted(frame, 1.0, object_trajectory, 0.5, 0)
        
        # Crop area
        frame = frame[top_pts[1]:bot_pts[1], top_pts[0]:bot_pts[0]]
    
        # Resize to save disk space
        frame = cv2.resize(frame, (480, 320),cv2.INTER_LINEAR)
        
        cv2.imwrite(saved_path, frame)
        
        # For writing HTMl file
        image_paths.append(filename)
        captions.append(filename)
        processed_frames.append(frame)
        
    writeHTML(os.path.join(CURR_SEQUENCE_DIR, 'result.html'), image_paths, captions)
    
print("Done in {}".format(time.time() - start))
cap.release()

  0%|          | 0/4 [00:00<?, ?it/s]

Processing VIRAT_S_000207


100%|██████████| 4/4 [00:06<00:00,  1.49s/it]

Done in 6.031045436859131


In [ ]:
# import imageio
# imageio.plugins.ffmpeg.download()
# from moviepy.editor import *
# # Generate new video clip
# new_clip = ImageSequenceClip(processed_frames, fps=fps)
# new_clip.write_videofile('processed_clip_car.mp4', fps=fps, codec='mpeg4', bitrate='1e12')